# All used functions

In [2]:
import pandas as pd
import numpy as np
import urllib.parse
import math
from sklearn.metrics import jaccard_score
import random
import networkx as nx
from pyvis.network import Network
import webbrowser
import time
from collections import Counter

In [5]:
def jaccard_similarity(active_user_series, train_set):
    intersection = np.logical_and(active_user_series, train_set)
    union = np.logical_or(active_user_series, train_set)
    return np.sum(intersection) / np.sum(union)

In [3]:
def GetIDFromName(name):
    data = pd.read_csv('ID_name_group_type_22_mei_2023.csv', index_col=0)
    name_df=data[data['name']==name]
    ID = name_df['ID'].iloc[0]
    return ID

In [ ]:
def GetNameFromID(ID):
    data = pd.read_csv('ID_name_group_type_22_mei_2023.csv', index_col=0)
    ID_df = data[data['ID']==ID]
    name = ID_df['name'].iloc[0]
    return name

In [4]:
def GetTypeFromName(name):
    data = pd.read_csv('ID_name_group_type_22_mei_2023.csv', index_col=0)
    name_df = data[data['name']==name]
    type = name_df['type'].iloc[0]
    return type

In [5]:
def GetTypeFromID(ID):
    data = pd.read_csv('ID_name_group_type_22_mei_2023.csv', index_col=0)
    ID_df = data[data['ID']==ID]
    type = ID_df['type'].iloc[0]
    return type

In [8]:
def GetMemberItems(df, member):
    all_items = []
    
    df_small = df[df.index == member]
    for i in range(len(df_small.columns)):
        if df_small[df_small.columns[i]].iloc[0] == 1:

            all_items.append(df_small.columns[i]) 
    
 
    return all_items   

In [13]:
def lastWord(string):
    lis = list(string.split(" "))
    length = len(lis)
    return lis[length-1]

In [15]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Function to make recommendation to users

In [4]:
def Recommendation(user_ID, t=0.175, top_n=5):
    '''
    Recommendation returns: user_ID, user_items, selected neighbors and recommended community's
    
    The model used to make a recommendation is user-based collaborative filtering which works in 3 steps:
        1) Calculate similarity with active users and all other users based on membership
        2) Select amount of similar users, the neighbors
        3) Calculate the weighted sum for each community and recommened the top_n community's
    
    The parameters:
        - t:     the amount of similar users you want to select. Default is 0.175
        - top_n: the amount of recommendations you would like. Default is 5
        !note it is possible that you don't have any recommendations or the actual amount doesn't meet your wishes
    '''

    df = pd.read_csv('Prediction matrix 22 mei 2023', index_col=0)
    all_columns = list(df.columns)
    
    #Step1) Get active user information
    user_dict = dict.fromkeys(all_columns, 0) 
    user_items = GetMemberItems(df, user_ID)
    for item in user_items:
        user_dict[item] = 1
    active_user = pd.DataFrame(data=[user_dict], columns=all_columns, index=[user_ID])
    active_user_series = active_user.iloc[0]
    other_users = df.drop(user_ID)
    
    #Step2) Calculate similarity between active user and all other users
    results = other_users.apply(lambda row: jaccard_similarity(active_user_series, row), axis=1) #calculate similarities

    results = results.sort_values(ascending=False)
    neighbors = results[(results.values > t) & (results.values < 1)].index
    scores = results[(results.values > t) & (results.values < 1)].values
    
    #Step3) calculate weighted sum for each community
    df_neighbors = df.loc[neighbors] 
    df_neighbors['weights'] = scores
    prediction_score = {}
    for w in range(len(df_neighbors.columns)-1): 
        if sum(df_neighbors['weights']) == 0:
            prediction_score[df_neighbors.columns[w]] = 0
        else:
            score = np.average(df_neighbors[df_neighbors.columns[w]], weights=df_neighbors['weights']) 
            prediction_score[df_neighbors.columns[w]] = score

    recommended_items = []

    prediction_sorted = dict(sorted(prediction_score.items(), key=lambda item: item[1], reverse=True))
    amount=0
    recommended = []
    
    #Step4) recommend top_n community's if not a hub, not in user_items and score higher then 0
    for f in prediction_sorted.keys(): 
        if (GetTypeFromID(f)!='hub') and (f not in user_items) and prediction_sorted[f] > 0:
            amount+=1
            recommended.append(f)
        if amount > (top_n-1):
            break
    
    print(color.BOLD + 'user',GetNameFromID(user_ID) + color.END)
    print(f'This user is member of {len(user_items)} communities')
    print('The recommended communities are:')
    for i in recommended:
        print('\t*',GetNameFromID(i), 'with score:', round(prediction_sorted[i],3))
    
    return user_ID, user_items, neighbors, recommended



## Function to make recommendations visual

In [2]:
def MakeVisual(user_ID, user_items, neighbors, recommended):
    '''
    MakeVisual returns: graph with the active user, its community's, the selected neighbors and the recommended community's
    The graph is saved as: graph.html
    
    The parameters:
        - user_ID:     the ID of the active users. This is also return by the Recommendation function.
        - user_items:  the community's of which the active user is member of. Return by Recommendation function.
        - neighbors:   the selected neighbors which shows most similarity with active user. Return by Recommendation function.
        - recommended: the recommended community's to the active user. Return by Recommendation function.
    '''
    
    df_open = pd.read_csv('data_graph_matrix_22_mei_2023.csv', index_col=0)
    df_select = df_open.loc[:,:]
    neighbors = neighbors.append(pd.Index([user_ID]))
    
    for a in range(len(df_select)-1,-1,-1): #select right items and users
        if df_select['source'].iloc[a] not in neighbors and df_select['target'].iloc[a] not in neighbors:
            df_select = df_select.drop(df_open.index[a])
            
    for b in range(len(df_select)): #add correct labels
        if df_select['target'].iloc[b] == user_ID:
            df_select['type Target'].iloc[b] = 'Active User'
        elif df_select['source'].iloc[b] == user_ID:
            df_select['type Source'].iloc[b] = 'Active User'

        elif df_select['target'].iloc[b] in neighbors[0:-1]:
            df_select['type Target'].iloc[b] = 'Selected Neighbor'
        elif df_select['source'].iloc[b] in neighbors[0:-1]:
            df_select['type Source'].iloc[b] = 'Selected Neighbor'

        if df_select['target'].iloc[b] in recommended:
            df_select['type Target'].iloc[b] = 'Recommended Community'
        elif df_select['source'].iloc[b] in recommended:
            df_select['type Source'].iloc[b] = 'Recommended Community'

        if df_select['target'].iloc[b] in user_items :
            df_select['type Target'].iloc[b] = 'Your Community'
        elif df_select['source'].iloc[b] in user_items :
            df_select['type Source'].iloc[b] = 'Your Community'
        
    OPEN_ID = nx.Graph()

    nodes = list(set(df_select['name Source']).union(set(df_select['name Target']))) #adding nodes
    for node in nodes:
        category1_matches = df_select['name Source'] == node
        category2_matches = df_select['name Target'] == node
        if category1_matches.any() or category2_matches.any():
            category1 = df_select.loc[category1_matches, 'type Source'].iloc[0] if category1_matches.any() else None
            category2 = df_select.loc[category2_matches, 'type Target'].iloc[0] if category2_matches.any() else None
            OPEN_ID.add_node(node, category1=category1, category2=category2)

    edges = [(row['name Source'], row['name Target']) for index, row in df_select.iterrows()] #adding edges
    OPEN_ID.add_edges_from(edges)
    colors = {'Active User':"red", 'Selected Neighbor':'blue', 'Your Community':'orange', 'Recommended Community':'magenta', 'hub':'black', 'challenge':'black', 'opportunity':'black', 'user':'black', 'organization':'black'}
    node_colors = [colors[OPEN_ID.nodes[node]['category1']] if OPEN_ID.nodes[node]['category1'] else colors[OPEN_ID.nodes[node]['category2']] for node in OPEN_ID.nodes()]

    nt = Network(height="1000px", width="1000px") #create network
    nt.barnes_hut(gravity=-2000, central_gravity=1.2, spring_length=40, spring_strength=0, damping=0.09, overlap=0) #makes sure you can move the nodes
    for i in range(len(nodes)): #add nodes to Network
        nt.add_node(nodes[i], label=nodes[i], value=20, color=node_colors[i], font='25px arial black')

    for x in range(len(edges)): #add edges to Network
        nt.add_edge(edges[x][0], edges[x][1])

    legend_items = [] # Create the legend
    other_communities = set()
    for category, color in colors.items():
        if color == 'black':
            other_communities.add(category)
        else:
            legend_items.append((category, color))

    if other_communities:
        legend_items.append(('Other Communities', 'black'))

    html_file = "graph.html"
    nt.save_graph(html_file)

    title_html = f'<h1 style="text-align: center;">Connectivity graph of {GetNameFromID(user_ID)}</h1>'
    legend_html = '<div style="position: absolute; top: 100px; left: 1025px; font-family: Arial, sans-serif; z-index: 9999;">'
    legend_html += '<h2 style="text-align: left;">Legend</h2>'
    for category, color in legend_items:
        legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 5px;">' \
                       f'<div style="width: 20px; height: 20px; background-color: {color}; margin-right: 10px;"></div>' \
                       f'<span style="font-size: 14px;">{category}</span>' \
                       f'</div>'
    legend_html += '</div>'

    with open(html_file, 'r+') as file:
        content = file.read()
        content = content.replace('<body>', f'<body>{title_html}{legend_html}')
        file.seek(0)
        file.write(content)
        file.truncate()

    webbrowser.open_new_tab(html_file)

In [12]:
def MakeVisualSimone(user_ID, user_items, neighbors, recommended):
    '''
    This MakeVisual Code includes the Alkemio Colours
    MakeVisual returns: graph with the active user, its community's, the selected neighbors and the recommended community's
    The graph is saved as: graph.html
    
    The parameters:
        - user_ID:     the ID of the active users. This is also return by the Recommendation function.
        - user_items:  the community's of which the active user is member of. Return by Recommendation function.
        - neighbors:   the selected neighbors which shows most similarity with active user. Return by Recommendation function.
        - recommended: the recommended community's to the active user. Return by Recommendation function.
    '''
    
    df_open = pd.read_csv('data_graph_matrix_22_mei_2023.csv', index_col=0)
    df_select = df_open.loc[:,:]
    neighbors = neighbors.append(pd.Index([user_ID]))
    
    for a in range(len(df_select)-1,-1,-1): #select right items and users
        if df_select['source'].iloc[a] not in neighbors and df_select['target'].iloc[a] not in neighbors:
            df_select = df_select.drop(df_open.index[a])
            
    for b in range(len(df_select)): #add correct labels
        if df_select['target'].iloc[b] == user_ID:
            df_select['type Target'].iloc[b] = 'Active User'
        elif df_select['source'].iloc[b] == user_ID:
            df_select['type Source'].iloc[b] = 'Active User'

        elif df_select['target'].iloc[b] in neighbors[0:-1]:
            df_select['type Target'].iloc[b] = 'Selected Neighbor'
        elif df_select['source'].iloc[b] in neighbors[0:-1]:
            df_select['type Source'].iloc[b] = 'Selected Neighbor'

        if df_select['target'].iloc[b] in recommended:
            df_select['type Target'].iloc[b] = 'Recommended Community'
        elif df_select['source'].iloc[b] in recommended:
            df_select['type Source'].iloc[b] = 'Recommended Community'

        if df_select['target'].iloc[b] in user_items :
            df_select['type Target'].iloc[b] = 'Your Community'
        elif df_select['source'].iloc[b] in user_items :
            df_select['type Source'].iloc[b] = 'Your Community'
        
    OPEN_ID = nx.Graph()

    nodes = list(set(df_select['name Source']).union(set(df_select['name Target']))) #adding nodes
    for node in nodes:
        category1_matches = df_select['name Source'] == node
        category2_matches = df_select['name Target'] == node
        if category1_matches.any() or category2_matches.any():
            category1 = df_select.loc[category1_matches, 'type Source'].iloc[0] if category1_matches.any() else None
            category2 = df_select.loc[category2_matches, 'type Target'].iloc[0] if category2_matches.any() else None
            OPEN_ID.add_node(node, category1=category1, category2=category2)

    edges = [(row['name Source'], row['name Target']) for index, row in df_select.iterrows()] #adding edges
    OPEN_ID.add_edges_from(edges, edge_color='lightblue')
    colors = {'Active User':"darkturquoise", 'Selected Neighbor':'darkcyan', 'Your Community':'bisque', 'Recommended Community':'darkorange', 'hub':'black', 'challenge':'black', 'opportunity':'black', 'user':'black', 'organization':'black'}
    node_colors = [colors[OPEN_ID.nodes[node]['category1']] if OPEN_ID.nodes[node]['category1'] else colors[OPEN_ID.nodes[node]['category2']] for node in OPEN_ID.nodes()]

       
    nodes_to_remove = []
    nodes_to_keep = []
    for node_r in range(len(nodes)):
        if node_colors[node_r] == 'black':
            nodes_to_remove.append(nodes[node_r])
        else:
            nodes_to_keep.append(nodes[node_r])
    OPEN_ID.remove_nodes_from(nodes_to_remove)
    
    filtered_colors = {'Active User':"darkorange", 'Selected Neighbor':'bisque', 'Your Community':'darkcyan', 'Recommended Community':'darkturquoise'}
    filtered_node_colors = [filtered_colors[OPEN_ID.nodes[node]['category1']] if OPEN_ID.nodes[node]['category1'] else filtered_colors[OPEN_ID.nodes[node]['category2']] for node in OPEN_ID.nodes()]
    legend_items = [] # Create the legend
    for category, color in filtered_colors.items():
        legend_items.append((category, color))

    
    fig = plt.figure(figsize=(25, 25))
    pos = nx.spring_layout(OPEN_ID)
    nx.draw_networkx_nodes(OPEN_ID, pos, node_size=400, node_color=filtered_node_colors)
    nx.draw_networkx_edges(OPEN_ID, pos, edge_color='lightblue')
    # Create a legend
    legend_handles = []
    for typ, color in filtered_colors.items():
        legend_handles.append(plt.Line2D([], [], color=color, marker='o', linestyle='', label=typ))
    plt.legend(handles=legend_handles, fontsize=16)

    plt.show()

    html_file = "graph without labels.html"

    nt = Network(height="1000px", width="1000px") #create network
    nt.barnes_hut(gravity=-2000, central_gravity=1.2, spring_length=40, spring_strength=0, damping=0.09, overlap=0) #makes sure you can move the nodes
    for i in range(len(nodes_to_keep)): #add nodes to Network
        nt.add_node(nodes_to_keep[i], label=nodes_to_keep[i], value=20, color=filtered_node_colors[i], font='25px arial black')


    for x in range(len(edges)): #add edges to Network
        if edges[x][0] in nodes_to_keep and edges[x][1] in nodes_to_keep:
            nt.add_edge(edges[x][0], edges[x][1], color='lightblue')
        else:
            continue

    legend_items = [] # Create the legend
    for category, color in filtered_colors.items():
        legend_items.append((category, color))


    nt.save_graph(html_file)

    title_html = f'<h1 style="text-align: center;">Connectivity graph of {GetNameFromID(user_ID)}</h1>'
    legend_html = '<div style="position: absolute; top: 100px; left: 1025px; font-family: Arial, sans-serif; z-index: 9999;">'
    legend_html += '<h2 style="text-align: left;">Legend</h2>'
    for category, color in legend_items:
        if color != 'black':
            legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 5px;">' \
                           f'<div style="width: 20px; height: 20px; background-color: {color}; margin-right: 10px;"></div>' \
                           f'<span style="font-size: 14px;">{category}</span>' \
                           f'</div>'
    legend_html += '</div>'

    with open(html_file, 'r+') as file:
        content = file.read()
        content = content.replace('<body>', f'<body>{title_html}{legend_html}')
        file.seek(0)
        file.write(content)
        file.truncate()

    webbrowser.open_new_tab(html_file)

## Function to make recommendations to community

In [ ]:
def RecommendtoCommunity(community_ID, amount_recommendations=10, amount_neighbors=50):
    print('The recommendations are based on the most similar contributors to the contributors in your community.\nThe similarity is calculated based on the communitys contributors are member of.\nPer member of the community the similarity between it and the contributors outside of the community is calculated.\nThe top 50 similar contributors are selected.\nSo every member of the community has a list of most similar users.\nThe contributors that are selected the most as similar contributor are being recommended to the community.\n')
    employees = pd.read_csv('employees and id.csv', index_col=0)
    users_per_com = []
    neighbors_per_community=[]
    all_user_items=[]
    for contributor in df.index:
        if df.loc[contributor,community_ID] ==1 :
            users_per_com.append(contributor)
    
    other_users = df.drop(users_per_com)
    for user in users_per_com:

        user_items = GetMemberItems(df, user)
        for user_item in user_items:
            all_user_items.append(user_item)
        og_user_dict = dict.fromkeys(df.columns, 0)
        for item in user_items:
            og_user_dict[item] = 1
        active_user = pd.DataFrame(data=[og_user_dict], columns=df.columns, index=[user])
        active_user_series = active_user.iloc[0]
        
        results = other_users.apply(lambda row: jaccard_similarity(active_user_series, row), axis=1)
        
        results = results[results > 0]  
        results = results.sort_values(ascending=False)  
        top = results.head(amount_neighbors)  

        neighbors = top.index  
        scores = top.values 
        
        neighbors_per_community.append(neighbors)
    

    neighbors_total = []
    for sublist in neighbors_per_community:
        neighbors_total.extend(sublist)
    neighbors_dict = dict(Counter(neighbors_total))
    neighbors_dict = dict(sorted(neighbors_dict.items(), key=lambda x: x[1], reverse=True))
    
    recommended_contributor=[]
    amount =0
    for value, item in neighbors_dict.items():
        if value not in users_per_com and value not in employees['ID'].to_list() and item>=1:
            recommended_contributor.append(value)
            amount+=1
        if amount == amount_recommendations:
            break

    print(color.BOLD + 'community',GetNameFromID(community_ID) + color.END, ',', GetTypeFromID(community_ID))
    print(f'This community has {len(users_per_com)} members')
    print('recommended contributors are:')
    for i in recommended_contributor:
        print('\t*',GetNameFromID(i), 'amount overlap:', neighbors_dict[i]) 
    return community_ID, users_per_com, all_user_items, neighbors_total, recommended_contributor
            

## Function to visualize recommendations to communities

In [1]:
def MakeVisualCommunity(community_ID, members, all_user_items, neighbors_total, recommended):
    '''
    MakeVisual returns:
    The graph is saved as: graph_community.html
    
    The parameters:

    '''
    
    df_open = pd.read_csv('data_graph_matrix_22_mei_2023.csv', index_col=0)
    df_select = df_open.loc[:,:]
    neighbors_total.extend(members)
    neighbors = neighbors_total
    
    for a in range(len(df_select)-1,-1,-1): #select right items and users
        if df_select['source'].iloc[a] not in neighbors and df_select['target'].iloc[a] not in neighbors:
            df_select = df_select.drop(df_open.index[a])
            
    for b in range(len(df_select)): #add correct labels

            
        if df_select['target'].iloc[b] in members:
            df_select['type Target'].iloc[b] = 'Community Member'
        elif df_select['source'].iloc[b] in members:
            df_select['type Source'].iloc[b] = 'Community Member'

        elif df_select['target'].iloc[b] in neighbors[0:-len(members)]:
            df_select['type Target'].iloc[b] = 'Selected Neighbor'
        elif df_select['source'].iloc[b] in neighbors[0:-len(members)]:
            df_select['type Source'].iloc[b] = 'Selected Neighbor'

        if df_select['target'].iloc[b] in recommended:
            df_select['type Target'].iloc[b] = 'Recommended Contributor'
        elif df_select['source'].iloc[b] in recommended:
            df_select['type Source'].iloc[b] = 'Recommended Contributor'

        if df_select['target'].iloc[b] in all_user_items :
            df_select['type Target'].iloc[b] = 'Member Community'
        elif df_select['source'].iloc[b] in all_user_items :
            df_select['type Source'].iloc[b] = 'Member Community'
        if df_select['target'].iloc[b] == community_ID:
            df_select['type Target'].iloc[b] = 'Active Community'
        elif df_select['source'].iloc[b] == community_ID:
            df_select['type Source'].iloc[b] = 'Active Community'
        
    OPEN_ID = nx.Graph()

    nodes = list(set(df_select['name Source']).union(set(df_select['name Target']))) #adding nodes
    for node in nodes:
        category1_matches = df_select['name Source'] == node
        category2_matches = df_select['name Target'] == node
        if category1_matches.any() or category2_matches.any():
            category1 = df_select.loc[category1_matches, 'type Source'].iloc[0] if category1_matches.any() else None
            category2 = df_select.loc[category2_matches, 'type Target'].iloc[0] if category2_matches.any() else None
            OPEN_ID.add_node(node, category1=category1, category2=category2)

    edges = [(row['name Source'], row['name Target']) for index, row in df_select.iterrows()] #adding edges
    OPEN_ID.add_edges_from(edges)
    colors = {'Active Community':"red", 'Community Member':'orange','Selected Neighbor':'blue', 'Member Community':'yellow', 'Recommended Contributor':'magenta', 'hub':'black', 'challenge':'black', 'opportunity':'black', 'user':'black', 'organization':'black'}
    node_colors = [colors[OPEN_ID.nodes[node]['category1']] if OPEN_ID.nodes[node]['category1'] else colors[OPEN_ID.nodes[node]['category2']] for node in OPEN_ID.nodes()]

    nt = Network(height="1000px", width="1000px") #create network
    nt.barnes_hut(gravity=-2000, central_gravity=1.2, spring_length=40, spring_strength=0, damping=0.09, overlap=0) #makes sure you can move the nodes
    for i in range(len(nodes)): #add nodes to Network
        nt.add_node(nodes[i], label=nodes[i], value=20, color=node_colors[i], font='25px arial black')

    for x in range(len(edges)): #add edges to Network
        nt.add_edge(edges[x][0], edges[x][1])

    legend_items = [] # Create the legend
    other_communities = set()
    for category, color in colors.items():
        if color == 'black':
            other_communities.add(category)
        else:
            legend_items.append((category, color))

    if other_communities:
        legend_items.append(('Other Communities', 'black'))

    html_file = "graph_community.html"
    nt.save_graph(html_file)

    title_html = f'<h1 style="text-align: center;">Connectivity graph of {GetNameFromID(community_ID)}</h1>'
    legend_html = '<div style="position: absolute; top: 100px; left: 1025px; font-family: Arial, sans-serif; z-index: 9999;">'
    legend_html += '<h2 style="text-align: left;">Legend</h2>'
    for category, color in legend_items:
        legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 5px;">' \
                       f'<div style="width: 20px; height: 20px; background-color: {color}; margin-right: 10px;"></div>' \
                       f'<span style="font-size: 14px;">{category}</span>' \
                       f'</div>'
    legend_html += '</div>'

    with open(html_file, 'r+') as file:
        content = file.read()
        content = content.replace('<body>', f'<body>{title_html}{legend_html}')
        file.seek(0)
        file.write(content)
        file.truncate()

    webbrowser.open_new_tab(html_file)

### Not so relevant functions

In [ ]:
def GetLink(item_ID, item_type):
    if item_type == 'challenge':
        df = pd.read_csv('challenge information id, name, type, url, location',index_col=0)
    
    elif item_type == 'opportunity':
        df = pd.read_csv('opportunity information id, name, type, url, location',index_col=0)
        
    elif item_type == 'hub':
        df = pd.read_csv('hub information id, name, type, url, location',index_col=0)
        
    link = df[df['id']==item_ID]['url'].iloc[0]
    link = urllib.parse.urlparse(link)
    formatted_url = link.geturl()

    print(formatted_url)

In [ ]:
def GetLocation(item_ID, item_type):
    if item_type == 'challenge':
        df = pd.read_csv('challenge information id, name, type, url, location',index_col=0)
    
    elif item_type == 'opportunity':
        df = pd.read_csv('opportunity information id, name, type, url, location',index_col=0)
    
    country = df[df['id']==item_ID]['country'].iloc[0]
    city = df[df['id']==item_ID]['city'].iloc[0]

    if isinstance(country, str) and isinstance(city, str):
        print(f'Country this community is based in: {country}\nCity this community is based in: {city}')
    elif isinstance(country, str) and not isinstance(city, str):
        print(f'Country this community is based in: {country}')
    elif not isinstance(country, str) and isinstance(city, str):
        print(f'City this community is based in: {city}')
    else:
        print('-')
        
        

In [ ]:
def GetItemInformation(item_ID):
    print('-------------------------------------------------------------------------')
    print(color.BOLD + '\nBasic information' + color.END)
    print(f'name of the community: {GetNameFromID(item_ID)}')
    print(f'the type of community: {GetTypeFromID(item_ID)}')
    parent = GetParent(item_ID)
    print(color.BOLD + '\nParent community'+ color.END)
    for i in parent:
        print(f'The community is part of the {GetNameFromID(i)} {GetTypeFromID(i)}')
    print(color.BOLD+'\nThe location'+ color.END)
    GetLocation(item_ID, GetTypeFromID(item_ID))
    print(color.BOLD + '\nThe link to this community:'+ color.END)
    GetLink(item_ID, GetTypeFromID(item_ID))
    print()